In [ ]:
import pandas as pd

data = pd.read_csv("C:/Projects/ADSDB-acero_troiani/project/src/data_management/landing_zone/temporal/Opendata_2021.csv", encoding="latin1")
d_base = pd.read_csv("C:/Projects/ADSDB-acero_troiani/project/src/data_management/landing_zone/temporal/Portaldades_2021.csv", encoding="latin1")

data_normalized = data.groupby("neighborhood").agg({
    "gini_index": "mean", 
    "80_20_rate": "mean",
    "gross_income": "mean",
    "avg_age": "mean"
}).reset_index()

d_reference = pd.merge(d_base, data_normalized, on="neighborhood")
d_reference = d_reference.round(2)

d1 = d_reference.drop(columns=["avg_rental_price", "gross_income", "avg_age"])
d1.to_csv("C:/Projects/FREYJA Demo/d_1.csv", index=False)

d_reference.drop(columns=["gini_index", "80_20_rate"]).to_csv("C:/Projects/FREYJA Demo/d_reference.csv", index=False)


In [ ]:
import pandas as pd

d_reference = pd.read_csv("C:/Projects/FREYJA Demo/d_reference.csv", encoding="latin1")
d_pop= pd.read_csv("C:/Projects/FREYJA Demo/2021_pad_mdbas.csv", encoding="latin1")

data_normalized = d_pop.groupby("neighborhood").agg({
    "population": "sum"
}).reset_index()

data_normalized

d_population = pd.merge(d_reference, data_normalized, on="neighborhood")
d_population.drop(columns=["avg_rental_price", "gross_income", "avg_age"]).to_csv("C:/Projects/FREYJA Demo/d_population.csv", index=False)

In [ ]:
d_reference = pd.read_csv("C:/Projects/FREYJA Demo/d_reference.csv", encoding="latin1")
d_sex= pd.read_csv("C:/Projects/FREYJA Demo/2021_pad_mdbas_sexe.csv", encoding="latin1")

aggregated_df = d_sex.groupby(['neighborhood', 'sex'])['population'].sum().reset_index()

aggregated_df

# Step 2: Calculate the total population for each neighborhood
total_population = aggregated_df.groupby('neighborhood')['population'].sum().reset_index()
total_population.rename(columns={'population': 'total_population'}, inplace=True)

# Step 3: Merge the total population with the aggregated data
aggregated_df = pd.merge(aggregated_df, total_population, on='neighborhood')

# Step 4: Calculate the percentage of male population
aggregated_df['female_percentage'] = aggregated_df.apply(
    lambda row: (row['population'] / row['total_population']) * 100 
    if row['sex'] == 2 else 0, axis=1
)

# Step 5: Filter the rows to show only the male population and their percentage
female_percentage_df = aggregated_df[aggregated_df['sex'] == 2][['neighborhood', 'female_percentage']]

female_percentage_df.to_csv("C:/Projects/FREYJA Demo/d_gender.csv", index=False)

In [ ]:
d_gender = pd.read_csv("C:/Projects/FREYJA Demo/d_gender.csv")
d_population = pd.read_csv("C:/Projects/FREYJA Demo/d_population.csv")

d_population_data = pd.merge(d_gender, d_population, on="neighborhood")

d_population_data.to_csv("C:/Projects/FREYJA Demo/d_2.csv", index=False)

In [ ]:
import pandas as pd
import random
import faker

d_ref = pd.read_csv("C:/Projects/FREYJA Demo/d_reference.csv", encoding="latin1")

# Create a Faker instance to generate fake product data
fake = faker.Faker()

# Number of products to generate
num_products = 10000

# Lists of possible categories and brands
types = ['Music CD', 'Book', 'Film', 'Painting']
product_names = d_ref["neighborhood"]

# Function to generate a synthetic product entry
def generate_product():
    if random.random() < 0.2:  # 20% chance to pick a product name from the predefined list
        product_name = random.choice(product_names)
    else:
        product_name = fake.bs()
    type = random.choice(types)
    author = fake.name()

    return {
        'product_name': product_name,
        'author': author,
        'type': type
    }

# Generate the synthetic dataset
products = [generate_product() for _ in range(num_products)]

# Create a DataFrame
df = pd.DataFrame(products)

# Show a sample of the dataset
df.to_csv("C:/Projects/FREYJA Demo/d_3.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Loading a CSV file into a pandas DataFrame
d_ref = pd.read_csv("C:/Projects/FREYJA Demo/d_reference.csv", encoding="latin1")
d_1 = pd.read_csv("C:/Projects/FREYJA Demo/d_1.csv", encoding="latin1")
d_2 = pd.read_csv("C:/Projects/FREYJA Demo/d_2.csv", encoding="latin1")

data_d1 = pd.merge(d_ref, d_1, on="neighborhood")
data_d2 = pd.merge(d_ref, d_2, on="neighborhood")
data_d1_d2 = pd.merge(d_ref, d_1, on="neighborhood")
data_d1_d2 = pd.merge(data_d1_d2, d_2, on="neighborhood")

data_variants = [d_ref, data_d1, data_d2, data_d1_d2]

for data in data_variants:
    # Separate the features (X) and the target variable (y)
    X = data.drop(columns=['avg_rental_price', 'neighborhood'])  # features
    y = data['avg_rental_price']  # target variable

    # Split the data into training and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Initialize the model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=0)

    # Perform cross-validation (using R-squared as the scoring metric)
    cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')  # 5-fold cross-validation

    # Print cross-validation results
    print(f"Mean neg_mean_squared_error: {cv_scores.mean()}")
    print(f"-----")

    # Train the model on the training data
    rf_model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f'Mean Absolute Error: {mae}')
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}')
    print(f'--------------------------------')